# Notebook for development of the script - 2

In [125]:
import random as rd
import numpy as np
import matplotlib.pyplot as plt
from Bio.Seq import *
from Bio.Alphabet.IUPAC import IUPACAmbiguousDNA, IUPACUnambiguousDNA
import reprlib
import time
from sys import getsizeof

In [68]:
class Genome(Seq):
    """Classe Genome"""
    def __init__(self, seq, circular=True):
        Seq.__init__(self, seq, alphabet=IUPACUnambiguousDNA())
        self.circular=circular
        
#     def __str__(self):
#         """To show the attributes and values of the instance"""
#         out=''
#         for key, value in self.__dict__.items():
#             out+='{:20s}  {}\n'.format(key, reprlib.repr(value))
#         return out

    def sequencing(self, read_length=100, reads_nb=5000):
        reads=[]
        for _ in range(reads_nb):
            start = rd.randint(0, len(self)-1)
            read_seq = self._data[start:min(start+100, len(self))] + self._data[0:max(start+100-len(self), 0)]
            read = Read(read_seq)
            reads.append(read)
        return reads


In [69]:
class Read(Seq):
    """Classe Read"""
    def __init__(self, seq, circular=True):
        Seq.__init__(self, seq, alphabet=IUPACUnambiguousDNA())
    
#     def __str__(self):
#         """To show the attributes and values of the instance"""
#         out=''
#         for key, value in self.__dict__.items():
#             out+='{:20s}  {}\n'.format(key, reprlib.repr(value))
#         return out
    
    def generate_kmers(self, kmers_length=30):
        """Returns a list of the k-mers included in the read"""
        kmers=[]
        for i in range(len(self) - kmers_length):
            kmer_seq = self._data[i:i+kmers_length]
            kmer = Kmer(kmer_seq)
            kmers.append(kmer)
        return kmers


In [70]:
class Kmer(Seq):
    """Classe Kmer"""
    def __init__(self, seq):
        Seq.__init__(self, seq, alphabet=IUPACUnambiguousDNA())
        self.prefix = seq[:-1]
        self.suffix = seq[1:]
    
#     def __str__(self):
#         """To show the attributes and values of the instance"""
#         out=''
#         for key, value in self.__dict__.items():
#             out+='{:20s}  {}\n'.format(key, reprlib.repr(value))
#         return out
    

In [71]:
class Graph:
    def __init__(self, km1mers):
        self.nodes = tuple(km1mers)
        self.matrix = np.zeros((len(self.nodes), len(self.nodes)))
        self.eulerian = None
    
    def fill_matrix(self, kmers):
        n = 0
        for kmer in kmers:
            print('K-mer {:5d}'.format(n), end='\r')
            i = self.nodes.index(kmer.prefix)
            j = self.nodes.index(kmer.suffix)
            self.matrix[i, j]+=1
            n+=1
    
    def test_eulerian(self):
        for i in range(len(self.nodes)):
            if self.matrix[i, :].sum() != self.matrix[:, i].sum():
                print("PROBLEM : This graph is not Eulerian.")
                self.eulerian = False
                break
        if self.eulerian != False:
            print('SUCCESS : This graph is Eulerian !!')
            self.eulerian = True
        

In [72]:
# Trouve 1 cycle
def find_eulerian_cycle():
    # attention, ce code ne gere pas les sommets connectés avec eux-memes
    cycle = []
    start, s = np.array(np.where(graph.matrix))[:,0]
    cycle.append(start)
    graph.matrix[start, s]-=1
    while s!= start:
        cycle.append(s)
        t = np.where(graph.matrix[s, :])[0][0]     # s comme sommet, t comme temporary
        graph.matrix[s, t]-=1
        s = t
    return cycle

In [73]:
# Test fin
def test_fin():
    if graph.matrix.sum()==0: return True
    else: return False

In [84]:
# Assemble les cycles
def assemble(cycle, tab):      # cycle a parcourir pour avancer 
                           # & tableau des cycles restant a parcourir
    for s in cycle:        # prenons un sommet du cycle
        seq.append(s)        # ajoutons le a la sequence
        for c in tab:          # puis testons s'il est également connecté à un des cycles restant
            if s in c:
                nextcycle = c[c.index(s)+1:] + c[:c.index(s)+1]     # Si oui, recadrons le prochain cycle pour que le parcours de la liste commence bien au début
                nexttab = tab[tab.index(c)+1:] + tab[:tab.index(c)]   # enlevons également ce cycle du tableau pour le prochain appel de la fonction
                assemble(nextcycle, nexttab)
                tab.remove(c)
                break
                

In [ ]:
# PROGRAMME PRINCIPAL
# Parameters
genome_length= 10000
read_length=100
reads_nb=5000
kmers_length=30

# Genome generation
genome= Genome(''.join(rd.choices(["A", "T", "G", "C"], k=genome_length)), circular=True)
# genome = Genome('ATGGCGTGCA')

# Reads generation
reads = genome.sequencing(read_length=read_length, reads_nb=reads_nb)

# K-mers generation
kmers=set()
for read in reads:
    kmers.update(read.generate_kmers(kmers_length=kmers_length))

# (K-1)-mers generation
km1mers=set()
for kmer in kmers:
    km1mers.update([kmer.prefix, kmer.suffix])
    
# Graph generation
graph = Graph(km1mers)
graph.fill_matrix(kmers)
graph.test_eulerian()
print(reprlib.repr(graph.nodes))

# Trouve tous les cycles
cycles = []
while test_fin() == False:
    cycles.append(find_eulerian_cycle())
print(str(len(cycles))+' cycles trouvés')

# Assemble les cycles
if len(cycles) == 1:
    seq = cycles[0]
else:
    seq=[]
    assemble(cycles[0], cycles[1:])
print(reprlib.repr(seq))



/home/etienne/miniconda3/lib/python3.6/site-packages/Bio/Seq.py:155: BiopythonWarning: Biopython Seq objects now use string comparison. Older versions of Biopython used object comparison. During this transition, please use hash(id(my_seq)) or my_dict[id(my_seq)] if you want the old behaviour, or use hash(str(my_seq)) or my_dict[str(my_seq)] for the new string hashing behaviour.
  "the new string hashing behaviour.", BiopythonWarning)


In [122]:
# Recréé la sequence
genome_assembly = ''
for s in seq:
    genome_assembly+=graph.nodes[s][0]
genome_assembly

# test genome de départ == genome assemblé
offset = (genome_assembly*2).index(genome._data)
genome_assembly[offset:]+genome_assembly[:offset] == genome._data

True

---
## Tests

In [ ]:
def assemble(cycle, tab):      # cycle a parcourir pour avancer 
                           # & tableau des cycles restant a parcourir
    for s in cycle:        # prenons un sommet du cycle
        seq.append(s)        # ajoutons le a la sequence
        for c in tab:          # puis testons s'il est également connecté à un des cycles restant
            if s in c:
                nextcycle = c[c.index(s)+1:] + c[:c.index(s)]     # Si oui, recadrons le prochain cycle pour que le parcours de la liste commence bien au début
                nexttab = tab[tab.index(c)+1:] + tab[:tab.index(c)]   # enlevons également ce cycle du tableau pour le prochain appel de la fonction
                
                

In [ ]:
def assemble(cycle, tab):      # cycle a parcourir pour avancer 
                           # & tableau des cycles restant a parcourir
    for s in cycle:        # prenons un sommet du cycle
        seq.append(s)        # ajoutons le a la sequence
        for j, c in enumerate(tab):          # puis testons s'il est également connecté à un des cycles restant
            for i, s1 in enumerate(c):
                if s == s1:
                    nextcycle = c[i+1:] + c[:i]     # Si oui, recadrons le prochain cycle pour que le parcours de la liste commence bien au début
                    nexttab = tab[j+1:] + tab[:j]   # enlevons également ce cycle du tableau pour le prochain appel de la fonction
            
            
            
            
            if s in c:
                nextcycle = c[c.index(s)+1:] + c[:c.index(s)]     # Si oui, recadrons le prochain cycle pour que le parcours de la liste commence bien au début
                nexttab = tab[j+1:] + tab[:j]   # enlevons également ce cycle du tableau pour le prochain appel de la fonction
                
                

In [158]:
getsizeof(np.zeros(12, dtype=bool))

108